In [1]:
from IPython.display import Image 

In [2]:
from ocean_lib.ocean.ocean import Ocean
from ocean_lib.config import Config

config = Config('config.ini')
ocean = Ocean(config)

print(f"config.network_url = '{config.network_url}'")
print(f"config.block_confirmations = {config.block_confirmations.value}")
print(f"config.metadata_cache_uri = '{config.metadata_cache_uri}'")
print(f"config.provider_url = '{config.provider_url}'")

config.network_url = 'https://rinkeby.infura.io/v3/d163c48816434b0bbb3ac3925d6c6c80'
config.block_confirmations = 0
config.metadata_cache_uri = 'https://aquarius.oceanprotocol.com'
config.provider_url = 'https://provider.rinkeby.oceanprotocol.com'


In [3]:
import os
from ocean_lib.web3_internal.wallet import Wallet

wallet = Wallet(ocean.web3, os.getenv('MY_TEST_KEY'), config.block_confirmations)

print(f"public address = '{wallet.address}'")

public address = '0xD438208197a0C552ED04e5e51695EC695E30C284'


In [5]:
from ocean_lib.web3_internal.currency import to_wei

# Publish ALG datatoken
ALG_datatoken = ocean.create_data_token('FASTCLASS2', 'FASTCLASS2', wallet, blob=ocean.config.metadata_cache_uri)
ALG_datatoken.mint(wallet.address, to_wei(100), wallet)
print(f"ALG_datatoken.address = '{ALG_datatoken.address}'")

ALG_datatoken.address = '0x1fDe09d7056F5A077e67C9170998855dbE0DE62D'


In [6]:
from ocean_lib.web3_internal.currency import pretty_ether_and_wei
print(f"Pool has {pretty_ether_and_wei(ALG_datatoken.balanceOf(wallet.address), ALG_datatoken.symbol())}.")

Pool has 1E+2 FASTCLASS2 (100000000000000000000 wei).


Ocean Protocol provides some basic Dockerfiles in their [profile](https://hub.docker.com/r/oceanprotocol/algo_dockers) on Dockerhub. In the previous tutorial, we uses the fast.ai library to train a simple image classifier, which isn't installed in the Ocean Protocol containers. Hence, we will use a Dockerfile from the fast.ai [profile](https://hub.docker.com/r/fastdotai/fastai/tags?page=1&ordering=last_updated) on Dockerhub. 

In [7]:
# Specify metadata and service attributes, for "GPR" algorithm script.
# In same location as Branin test dataset. GPR = Gaussian Process Regression.
ALG_metadata =  {
    "main": {
        "type": "algorithm",
        "algorithm": {
            "language": "python",
            "format": "docker-image",
            "version": "0.11",
            "container": {
              "entrypoint": "python $ALGO",
              "image": "fastdotai/fastai",
              "tag": "latest"
            }
        },
        "files": [
    {
        "url": "https://raw.githubusercontent.com/AlgoveraAI/onshore/main/fast-classification.py",
        "index": 0,
        "contentType": "text/text",
      }
    ],
    "name": "fast-classification-v0.11", "author": "AlgoveraAI", "license": "CC0",
    "dateCreated": "2020-10-05T12:42:00Z"
    }
}

In [8]:
ALG_service_attributes = {
        "main": {
            "name": "ALG_dataAssetAccessServiceAgreement",
            "creator": wallet.address,
            "timeout": 3600 * 24,
            "datePublished": "2020-01-28T10:55:11Z",
            "cost": 1.0, # <don't change, this is obsolete>
        }
    }

In [9]:
from ocean_lib.common.agreements.service_factory import ServiceDescriptor
# Set up a service provider. We'll use this same provider for ALG
from ocean_lib.data_provider.data_service_provider import DataServiceProvider
provider_url = DataServiceProvider.get_url(ocean.config)

# Calc ALG service access descriptor. We use the same service provider as DATA
ALG_access_service_descriptor = ServiceDescriptor.access_service_descriptor(ALG_service_attributes, provider_url)
#returns ("algorithm",
#         {"attributes": ALG_service_attributes, "serviceEndpoint": provider_url})

In [10]:
# Publish metadata and service info on-chain
ALG_ddo = ocean.assets.create(
  metadata=ALG_metadata, # {"main" : {"type" : "algorithm", ..}, ..}
  publisher_wallet=wallet,
  service_descriptors=[ALG_access_service_descriptor],
  data_token_address=ALG_datatoken.address)
print(f"ALG did = '{ALG_ddo.did}'")

signing message with nonce 16: 0x1fDe09d7056F5A077e67C9170998855dbE0DE62D, account=0xD438208197a0C552ED04e5e51695EC695E30C284
ALG did = 'did:op:1fDe09d7056F5A077e67C9170998855dbE0DE62D'


At this point you will need to request that the data provider approves your code as a trusted algorithm on the dataset. For example, if you reach out to Algovera about one of our datasets, we will approve your request as quickly as possible. After your algorithm is approved for a dataset, it will be displayed on the page for the dataset, as shown in the image below. 

In [ ]:
display(Image(filename='images/algo-approved.png', width = 800))